In [86]:
import torch.nn as nn

In [97]:
random_tensor = torch.randint(1, 256, (1,3, 1, 640, 640), dtype=torch.float32)

In [98]:
random_tensor.shape

torch.Size([1, 3, 1, 640, 640])

In [99]:
class Backbone(nn.Module):
    def __init__(self):
        super(Backbone, self).__init__()

        self.p1 = nn.Conv3d(in_channels=3, out_channels=16, kernel_size=3, padding=1, stride=2).to(dtype=torch.float32)
        self.p2 = nn.Conv3d(in_channels=16, out_channels=32, kernel_size=3, padding=1, stride=2).to(dtype=torch.float32)
        
        '''
        TODO : Create C2f
        '''
        
        self.p3 =  nn.Con3d(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=2).to(dtype=torch.float32)

        
    def forward(self, x):
        x = self.p1(x)
        x = self.p2(x)
        x = self.p3(x)
        return x

In [132]:
class Bottleneck(nn.Module):
    def __init__(self,shortcut,in_channels):
        super(Bottleneck, self).__init__()
        self.conv1_bottleneck = nn.Conv3d(in_channels=in_channels, out_channels=in_channels, kernel_size=3, padding=1, stride=1).to(dtype=torch.float32)
        self.conv2_bottleneck = nn.Conv3d(in_channels=in_channels, out_channels=in_channels, kernel_size=3, padding=1, stride=1).to(dtype=torch.float32)
        self.shortcut = shortcut


    def forward(self, x):
        y = x.clone()
        x = self.conv1_bottleneck(x)
        x = self.conv2_bottleneck(x)
        
        if self.shortcut: 
            x = x + y
            
        return x

In [133]:
class C2f(nn.Module):
    def __init__(self,n,shortcut,in_channels):
        super(C2f, self).__init__()
        self.n = n
        self.shortcut = shortcut
        self.conv1 = nn.Conv3d(in_channels=in_channels, out_channels=in_channels, kernel_size=1, padding=0, stride=1).to(dtype=torch.float32)
        self.bottleneck_blocks = nn.ModuleList([Bottleneck(self.shortcut, in_channels / 2) for _ in range(self.n)])
        self.conv2 = nn.Conv3d(in_channels=in_channels, out_channels= (1/2) * (self.n+2)*in_channels, kernel_size=1, padding=0, stride=1).to(dtype=torch.float32)

    def forward(self, x):
        x = self.conv1_c2f(x)
        x1, x2 = torch.chunk(x, 2, dim=1)
        x2 = torch.cat((x2, x1), dim=1)

        for bottleneck_block in self.bottleneck_blocks:
             x1 = bottleneck_block(x1)
             x2 = torch.cat((x2, x1), dim=1)
        
        x = x2
        x = self.conv2(x)
        return x

In [134]:
model = C2f(n=1,shortcut=True,in_channels=32)

TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [122]:
random_tensor = torch.randint(1, 256, (1,32, 1, 160, 160), dtype=torch.float32)

In [123]:
output = model(random_tensor)

In [124]:
list(output.shape)

[1, 32, 1, 640, 640]

In [126]:
assert list(output.shape) ==  [1 ,32 ,1 ,640 ,640]

In [127]:
x1, x2 = torch.chunk(output, 2, dim=1)